In [ ]:
using PyCall
using Plots
using Images
using MLDatasets
using Zygote
using FFTW
using PaddedViews
using Statistics

@pyimport sklearn.datasets as datasets
@pyimport matplotlib.pyplot as plt

In [ ]:

size(x)

In [ ]:
train_x, train_y = MNIST.traindata();

test_x, test_y = MNIST.testdata();

train_x = permutedims(train_x, [3,2,1]);

test_x = permutedims(test_x, [3,2,1]);


In [ ]:
x, y = datasets.load_digits(return_X_y=true)

val_x = reshape(x[1:256,:],256, 8, 8);
val_y = get_one_hot(y[1:256]);

train_x = reshape(x[257:1280, :], 1024, 8, 8);
train_y = get_one_hot(y[256:1280]);

test_x = reshape(x[1281:1797, :], 517, 8, 8);
test_y = get_one_hot(y[1280:1797]);

train_x = PaddedView(0.0, train_x, (1024, 64,64), (1, 31, 31));
test_x = PaddedView(0.0, test_x, (517, 64,64), (1, 31, 31));
val_x = PaddedView(0.0, test_x, (256, 64,64), (1, 31, 31));

In [ ]:
function asm_prop(wavefront, fxx, fyy, my_length=32.0e-3, wavelength=550.0e-9, distance=10.0e-3)
    
    dim_x, dim_y = size(wavefront)[2:3]
    
    if dim_x != dim_y
        println("Wavefront should be square")
    end
    
    px = my_length / dim_x
    
    l2 = (1/wavelength)^2 
    
    #meshgrid(xx,xx)    
    
    q = l2 .- fxx.^2 .- fyy.^2
    
    q = reshape(q, 1, dim_x, dim_y)
    
    # no mutating arrays with Zygote, so make a new array
    q2 = (q.>= 0.0) .* q
           
    h = fftshift(exp.(im * 2 * π * distance * sqrt.(q2)), [2,3])
    
    fd_wavefront = fft(fftshift(wavefront, [2,3]), [2,3])
    
    fd_new_wavefront = h .* fd_wavefront
     
    new_wavefront = ifftshift(ifft(fd_new_wavefront, [1,2]), [1,2]) #[:dim_x,:dim_x]
    
end

function meshgrid(x, y)
    xx = [ii for ii in x, jj in 1:length(y)]
    yy = [jj for ii in 1:length(x), jj in y]
    return xx, yy
end

function kerr_effect(wavefront, n2=1e-20)
   return exp.(im .* abs.(wavefront.^2) * n2) 
end

function softmax(x)
    
    x = x .- maximum(x, dims=2) .+ 1.e-3

    (exp.(x)) ./ sum(exp.(x), dims=[2])
    
end

function get_accuracy(y, pred, boundary=0.5) 
    temp = [elem[2] for elem in argmax(y,dims=2)]
    temp2 = [elem[2] for elem in argmax(pred,dims=2)]
    

    return mean(temp .== temp2) 
end

log_loss = function(y_tgt, pred)
   
    return -(1 / size(y_tgt)[1]) .* sum(y_tgt .* log.(pred) .+ (1.0 .- y_tgt) .* log.(1.0 .- pred))

end

function get_one_hot(tgt_y)
    
    batch_size, num_classes = size(tgt_y)[1], 10
    one_hot_y = zeros(batch_size, num_classes)
    [one_hot_y[hh, tgt_y[hh]+1] = 1.0 for hh in 1:batch_size]
    
    return one_hot_y 
    
end

function multiply_and_propagate(wavefront, phase_slices, fxx, fyy, n2=1e-20)
    
    for slice_idx in 1:length(phase_slices)
    
        wavefront = asm_prop(wavefront .* (phase_slices[slice_idx] .*
                kerr_effect(wavefront, n2)), fxx, fyy)
    
    end
        
    #wavefront = asm_prop(wavefront .* (phase_slices[:2] .* kerr_effect(wavefront, n2)))
    #wavefront = asm_prop(wavefront .* (phase_slices[:3] .* kerr_effect(wavefront, n2)))
    
    
    #end
    
    return wavefront
    
end

function optical_network(in_x, phase_slices, fxx, fyy, n2=1e-20)
    
    dim_n, dim_x, dim_y = size(in_x)
    
    if dim_x != dim_y
        println("Wavefront should be square")
    end
    
    start_x = Int(round(dim_x//2));
    stop_x = Int(round(3dim_x//2))-1;
    start_y = Int(round(dim_y//2));
    stop_y = Int(round(3dim_y//2))-1;
   
    #hard-coded padding, TODO: generalize for variable dimensions
    #wavefront = PaddedView(0.0 + 0.0 * im, data_array, (dim_n, 56,56), (1, 15,15))
    wavefront = ones(dim_n, dim_x, dim_y) .* 
                exp.(im*2π .* in_x) #.*
                #reshape((1.0 .* sqrt.(fxx.^2 + fyy.^2) .<= 8),1,dim_x,dim_y)
    
    wavefront2 = multiply_and_propagate(wavefront, phase_slices, fxx, fyy, n2)
    
    return wavefront2
    
end

function get_decision_zones(dim_n=4, dim_x=28, dim_y=28, num_classes = 10)
    
    # assuming a 10 class classifier (i.e. a digits dataset)
    xx, yy = meshgrid(1:dim_x, 1:dim_y)
    
    center_coords = zeros(16,2)
    for ii in 1:4
        for jj in 1:4
            center_coords[4*(ii-1) + jj,:] = [(ii*5) + 2, (jj*5) + 2]
        end
    end
    
    #center_coords = center_coords[3:12,:]
    
    decision_zones = zeros(dim_x, dim_y, 10)
    
    dec_r = 2.5 
      
    for ll in 1:10
        
        decision_zone = sqrt.((xx .- center_coords[ll,1]).^2 .+ (yy .- center_coords[ll,2]).^2)
        
        decision_zone[decision_zone .<= dec_r] .= 1.0 
        decision_zone[decision_zone .> dec_r] .*= 0.0 
        
        decision_zones[:,:,ll] .= decision_zone #./ (1.0e-5 .+ decision_zone)
        
    end
    
    return decision_zones
    
end

function get_pred(I, dz)
    
    dim_n = size(I)[1]
    
    
    temp = reshape(I,size(I)[1], size(I)[2], size(I)[3], 1) .* 
        reshape(dz, 1, size(dz)[1], size(dz)[2], size(dz)[3])
    

    pred = reshape(sum(temp, dims=[2,3]), size(I)[1], size(dz)[3])
 
end

function get_onn_loss(in_x, phase_slices, tgt_y, decision_zones, fxx, fyy, n2=1e-20)
    
    #wavefront = optical_network(in_x, phase_slices, fxx, fyy, n2)
    
    #dim_n, dim_x, dim_y = size(wavefront)
    
    #my_length = 32.e-3
    #px = my_length / dim_x
    
    #xx = range(-1/(2*px), stop=1/(2*px) - 1/(dim_x*px), length=dim_x)
    
    #xx, yy = meshgrid(1:dim_x, 1:dim_y)
    
    intensity = get_intensity(in_x, phase_slices, fxx, fyy, n2) #abs.(wavefront.^2)
    
    intensity2 = intensity ./ maximum(intensity, dims=[2,3])
    
    pred = get_pred(intensity2, decision_zones) 
    
    pred_sm = softmax(pred)
    
    loss = log_loss(tgt_y, pred_sm)
    #accuracy = get_accuracy(tgt_y, pred_sm)
    #println(accuracy)
    return loss    
      
end

function get_intensity(in_x, phase_slices, fxx, fyy, n2 = 1e-20)
    
    wavefront = optical_network(in_x, phase_slices, fxx, fyy, n2)
    
    intensity = abs.(wavefront.^2)
    return intensity
    
end



In [ ]:
batch_size, dim_x, dim_y = 32, 64, 64 #56, 56;


#phase_slices = [exp.(im * 2π * randn(1,dim_x, dim_y)) for aa in 1:num_slices];

#phase_slices = Dict(:1 => randn(1,dim_x, dim_y), 
#    :2 => randn(1,dim_x, dim_y),
#    :3 => 

num_slices = 8
phase_slices = [1.0 .* exp.((im * 2π) .* zeros(1 ,dim_x, dim_y)/100)
    for ii in 1:num_slices]


in_x = train_x[1:batch_size, :, :]

wavefront = 1e6 * exp.(im * 2π * randn(32,56,56))

#test1 = multiply_and_propagate(wavefront, phase_slices);
#test2 = optical_network(in_x, phase_slices);


decision_zones = get_decision_zones(batch_size, dim_x, dim_y, 10)
xx, yy = meshgrid(1:dim_x, 1:dim_y)

my_length = 32.e-3
px = my_length / dim_x

xx = range(-1/(2*px), stop=1/(2*px) - 1/(dim_x*px), length=dim_x)

fxx, fyy = meshgrid(xx,xx)

#frr = sqrt(fxx.^2 + fyy.^2)


lr = 1e-3
n2 = 1e-20
println("go already")
for step in 1:500
    
    #loss = get_onn_loss(in_x, phase_slices, y_one_hot[1:batch_size,:]);
    #temp 
    loss =  get_onn_loss(in_x, phase_slices, train_y[1:batch_size,:], decision_zones, fxx, fyy);
   
    if (step-1) % 50 == 0
        
        intensity = get_intensity(in_x, phase_slices, fxx, fyy, n2)
        intensity2 = intensity ./ maximum(intensity, dims=[2,3])
        pred = get_pred(intensity2, decision_zones)
        pred_sm = softmax(pred)
        accuracy = get_accuracy(train_y[1:batch_size,:], pred_sm)
        
        println("step $step, loss = $loss, accuracy = $accuracy"); #, size(d_slices))
        
    end

    d_slices = gradient((phase_slices) -> 
        get_onn_loss(in_x, phase_slices, train_y[1:batch_size,:], decision_zones, fxx, fyy),
        phase_slices);
    
    for slice_idx in 1:length(phase_slices)
        phase_slices[slice_idx] .*= - exp.(im .* angle.(lr .* d_slices[1][slice_idx]));
    end

#println()
    
end

In [ ]:
size(fxx)

In [ ]:
println(size(phase_slices[1]))
plt.figure()
plt.subplot(121)
plt.imshow(angle.(phase_slices[1][1,:,:]))
plt.subplot(122)
plt.imshow(abs2.(phase_slices[1][1,:,:]))
plt.show()

In [ ]:
plt.subplot(121)
plt.imshow(train_x[10,:,:])
plt.subplot(122)
plt.imshow(test_x[10,:,:])
plt.show()

In [ ]:
temp = randn(2,2)
temp2 = randn(2,2)
temp3 = [temp .> 0] .* temp2

println(temp)
println(" ")
println(temp2)
println(" ")
println([temp .> 0])
println(" ")
println( temp .* temp2)
println(" ")
println((temp .> 0) .* temp2)

In [ ]:
using PaddedViews

dim_x = 28

temp = PaddedView(0.0, randn(28,28), (dim_x,56), (1,15)  )

plt.figure()
plt.imshow(temp) #[1:42,1:42])
plt.show()

println(size(temp))

In [ ]:
length(d_slices[1][2]), size(d_slices[1][2])

In [ ]:

decision_zones = get_decision_zones(batch_size, dim_x, dim_y, 10)

println(size(decision_zones))

for bb in 1:10
    plt.figure()
    plt.imshow(decision_zones[:,:,bb])
end
plt.show()

In [ ]:
(12544 / 4) / 56

In [ ]:
temp = Zygote.gradient(get_onn_loss, in_x, phase_slices, y_one_hot[1:4,:], decision_zones,xx,yy,fxx,fyy)

println(length(temp))
println(temp[1], temp[2], size(temp[3]), size(temp[4]))#, size(temp[8])) #,
    #size(temp[6]))#, size(temp[7]))# size(temp[8]))

In [ ]:

function get_pred(intensity2, decision_zones)
    
    dim_n = size(intensity2)[1]
    
    [sum(decision_zones[:,:,ll] .* intensity2[mm,:,:]) for mm in 1:dim_n, ll in 1:10]
    
end

I = randn(4,56,56)/10
dz = randn(56,56,10)/10

temp = reshape(I,size(I)[1], size(I)[2], size(I)[3], 1) .* 
    reshape(dz, 1, size(dz)[1], size(dz)[2], size(dz)[3])
    
println(size(temp))

temp2 = reshape(sum(temp, dims=[2,3]), size(I)[1], size(dz)[3])

println(size(temp2))

pred_sm = softmax(temp2)

println(size(pred_sm))

loss = log_loss(y_one_hot[1:4,:], pred_sm)

println(pred_sm)
println(loss)

In [ ]:
? sum

In [ ]:
dz = randn(56,56,10)
I = randn(4,56,56)
dim_n = 4

pred = [ sum(dz[:,:,ll] .* I[mm,:,:]) for mm in 1:dim_n, ll in 1:10]

print(size(pred))

pred_sm = softmax(pred)

In [ ]:
decision_zones = get_decision_zones()
println(size(decision_zones))

for zone_idx in 1:10
    plt.figure()
    plt.imshow(decision_zones[:,:,zone_idx])
end
    
plt.show()

In [ ]:
phase_slices

In [ ]:
d_slices

In [ ]:
temp = softmax(test3);

temp[temp .== 0.0] .= 1.e-3


In [ ]:

batch_size, dim_x, dim_y = 32, 56, 56;
num_slices = 3;

y_one_hot = get_one_hot(train_y);
phase_slices = [exp.(im * 2π * randn(1,dim_x, dim_y)) for aa in 1:num_slices];


in_x = train_x[1:batch_size, :, :]

test = optical_network(in_x, phase_slices) #get_onn_loss(in_x, phase_slices, y_one_hot)

In [ ]:
x = randn(2,3)
y_idx = [elem[2] for elem in argmax(randn(2,3),dims=2)]
y = zeros(2,3)
[[y[hh,idx] = 1.0 for idx in y_idx[hh]] for hh in 1:size(y_idx)[1]]

print(y)

function softmax(x)
    
    x = x .- maximum(x)

    (exp.(x)) ./ sum(exp.(x), dims=[2])
    
end

function log_loss(y_tgt, pred)
   
    return -(1 / size(y_tgt)[1]) .* sum(y_tgt .* log.(pred) .+ (1.0 .- y_tgt) .* log.(1.0 .- pred))

end

res = sum(softmax(randn(2,3)), dims=2)

log_loss(y, res)

In [ ]:

print(softmax(x), y)

In [ ]:
in_x = train_x[1:10, :, :] / 255

dim_n, dim_x, dim_y = size(in_x)
println(dim_n, dim_x, dim_y)

wavefront = ones(dim_n, dim_x*2, dim_y*2) .+ (0.0 * im);

start_x = Int(round(dim_x//2));
stop_x = Int(round(3dim_x//2))-1;

wavefront[:, start_x:stop_x, start_x:stop_x] .*= exp.(im * 2π .* in_x)

plt.figure()
plt.imshow(angle.(wavefront[1,:,:]))
plt.show()

In [ ]:
size(wavefront[:, Int(round(dim_x//2)):Int(round(3dim_x//2))-1, Int(round(dim_y//2)):Int(round(3dim_y//2))-1])

In [ ]:
asm_prop = temp = zeros(1,28,28)
temp[:,13:15,13:15] .+= 1

wf = asm_prop(exp.(im .* train_x[1,:,:])) #, distance=1000.e-3)

for ii =1:500
    wf = asm_prop(wf) #, distance=1000.e-3)
end

img = abs2.(wf)

plt.subplot(121)
plt.imshow(train_x[1,:,:])
plt.subplot(122)
plt.imshow(img[:,:])
plt.show()



In [ ]:

center_coords = zeros(16,2)
for ii in 1:4
    for jj in 1:4
        center_coords[4*(ii-1) + jj,:] = [(ii*12) + 8, (jj*12) + 8]
    end
end

#print(size(center_coords))
center_coords